In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd SE/EasyToken
%ls

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd

In [ ]:
USAGE_files = []
usage_keywords = ['res','resources','ui','Activity']
for u in usage_keywords:
  for root, dirs, files in os.walk('/content/gdrive/MyDrive/SE/EasyToken'):
    for file in files:
      if u in root and file.endswith(('.java', '.xml')):
        USAGE_files.append(os.path.join(root,file))

USAGE_files

['/content/gdrive/MyDrive/SE/EasyToken/res/values-v14/styles.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/xml/widget_info.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/xml/settings.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/menu/main.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/animator/fragment_slide_right_exit.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/animator/fragment_slide_left_enter.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/animator/fragment_slide_right_enter.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/animator/fade_in.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/animator/fragment_slide_left_exit.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/animator/fade_out.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/values/dimens.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/values/strings.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/values/styles.xml',
 '/content/gdrive/MyDrive/SE/EasyToken/res/values/arrays.xml',
 '/content/gdrive/MyDrive

In [ ]:
Compatibility_Protection_files = []

for root, dirs, files in os.walk('/content/gdrive/MyDrive/SE/EasyToken'):
  for file in files:
    if 'AndroidManifest' in os.path.join(root,file):
      Compatibility_Protection_files.append(os.path.join(root,file))

Compatibility_Protection_files

['/content/gdrive/MyDrive/SE/EasyToken/AndroidManifest.xml']

In [ ]:
resources_keywords = ['content','provider','Content-Provider']
resources_files = []
for r in resources_keywords:
  for root, dirs, files in os.walk('/content/gdrive/MyDrive/SE/EasyToken'):
    for file in files:
      if r in root and file.endswith(('.java', '.xml')):
        resources_files.append(os.path.join(root,file))

resources_files

In [ ]:
data = pd.read_csv(USAGE_files[0],on_bad_lines='skip')
data.head()

In [ ]:
data = pd.read_csv("ALauncher.txt",on_bad_lines='skip')
data['package a2dp.Vol;']

In [ ]:
# Data Cleaning
import re
def clean_text(text):
    # remove everything except alphabets
    text = re.sub("[^a-zA-Z]", " ", text)
    # use camel split
    #text = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', text)
    # remove whitespaces
    text = ' '.join(text.split())
    text = text.lower()
    
    return text

In [ ]:
# creating clean text feature
features = ['package a2dp.Vol;']
for feature in features:
    data['clean_' + feature] = data[feature].apply(clean_text)

In [ ]:
data.head()

,package a2dp.Vol;,clean_package a2dp.Vol;
0,import java.io.FileInputStream;,import java io fileinputstream
1,import java.io.FileNotFoundException;,import java io filenotfoundexception
2,import java.io.IOException;,import java io ioexception
3,import android.app.Service;,import android app service
4,import android.content.Intent;,import android content intent


In [ ]:
data['soup'] = data['clean_package a2dp.Vol;']

In [ ]:
# stopwords-to compare text data with and without stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
java_stop_words = ['abstract','continue','for','new','switch','assert','default','goto','package','synchronized',
'boolean','do','if','private','this','break','double','implements','protected','throw','byte','else','import','public','throws',
'case','enum','instanceof',	'return','transient','catch','extends','int','short','try', 'char','final','interface',
'static',	'void','class',	'finally','long',	'strictfp',	'volatile', 'const','float',	'native',	'super','while']

for s in java_stop_words:
  stop_words.add(s)

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data['soup'] = data['soup'].apply(lambda x: remove_stopwords(x))

In [ ]:
# stemming process
from nltk.stem import PorterStemmer
'''
Words in the third person are changed to first person and verbs in past and future tenses are changed into the present by the 
stemming process. 
'''
stem_process = PorterStemmer()

def tokenize_and_stem(text):
    # tokenization to ensure that punctuation is caught as its own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    lem = [stem_process.stem(t) for t in filtered_tokens]
    return lem

In [ ]:
# Defining a TF-IDF Vectorizer with Ngram 2 and 3
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(2, 3), tokenizer=tokenize_and_stem, max_features=1000, use_idf=True)

In [ ]:
nltk.download('punkt')
# tfidf_vec = tfidf_vec.fit_transform(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
all_files = USAGE_files + Compatibility_Protection_files + resources_files
source_codes_tf_idf = []

# todo: check this part
for path in all_files:
  with open(path, 'r') as f:
    data = f.read()
    data = clean_text(data)
    data = [remove_stopwords(data)]
    data = tfidf_vec.fit(data)
    source_codes_tf_idf.append(data)
  

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pat

In [ ]:
reviews_tf_idf = []
# to do: check
reviews = pd.read_csv('/content/gdrive/MyDrive/SE/appbot-reviews.csv')
reviews_tf_idf = tfidf_vec.fit_transform(reviews['Body'])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

for r in reviews_tf_idf:
  cosine_similarities = []
  for t in source_codes_tf_idf:
    cosine_sim = cosine_similarity(t, r)
    # todo: boost cosine_sim with Lucene boosting functionality
    # we have reviews['label] and HLC of source_codes (USAGE_files, Compatibility_Protection_files, resources_files)
    # cosine_sim = ...
    cosine_similarities.append(cosine_sim)
  
  # todo: sort the cosine_similarities
  # todo: print top scoring source_codes for review r
